In [ ]:
import pandas as pd
import numpy as np
import pymongo
import sys
import os
from __future__ import print_function
from datetime import datetime

TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)

import db.etl as etl

In [ ]:
sids=[r['dsstox_sid'] for r in dsstox.find({},{'_id':0,'dsstox_sid':1})]

In [ ]:
study_types=predictions.distinct('study')

In [ ]:
predictions.find_one()

In [ ]:
pipeline=[
    {'$group':{
        '_id': {'dsstox_sid':'$dsstox_sid','study_type':'$study'},
        'average_ap':{'$avg':'$a_p'}
    }}
]

In [ ]:
agg=predictions.aggregate(pipeline,allowDiskUse=True)

In [ ]:
agg_list=[]
for result in agg:
    dsstox_sid=result['_id'].get('dsstox_sid',None)
    study_type=result['_id'].get('study_type',None)
    if dsstox_sid is None or study_type is None:
        continue
    average_ap=result['average_ap']
    agg_list.append({'dsstox_sid':dsstox_sid,'study_type':study_type,'average_ap':average_ap})

In [ ]:
df=pd.DataFrame(list(agg_list))

In [ ]:
df.head()

<h1>EDA</h1>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
chr_df=df[df['study_type']=='CHR']

In [ ]:
plt.hist(chr_df['average_ap'])
plt.xlabel('average a_p')
plt.title('CHR')

In [ ]:
for study_type in df['study_type'].unique():
    histogram=plt.hist(df[df['study_type']==study_type]['average_ap'])
    label=plt.xlabel('average a_p')
    title=plt.title(study_type)
    plt.savefig(FIG_DIR+'study-aggregation/' + study_type + '.png')
    plt.show()

In [ ]:
len(df[df['average_ap']==0])

In [ ]:
len(df)

In [ ]:
DB['tox_fp'].find_one()